<a href="https://colab.research.google.com/github/edenau/maps/blob/master/map_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install

!pip install folium==0.7.0
!pip install selenium

    100% |████████████████████████████████| 92kB 2.7MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.7.0 which is incompatible.
  Found existing installation: folium 0.2.1
    Uninstalling folium-0.2.1:
      Successfully uninstalled folium-0.2.1
    100% |████████████████████████████████| 911kB 17.4MB/s 


In [2]:
# Check OS

!cat /proc/version

Linux version 4.14.65+ (chrome-bot@swarm-cros-641) (gcc version 4.9.x 20150123 (prerelease) (4.9.2_cos_gg_4.9.2-r193-ac6128e0a17a52f011797f33ac3e7d6273a9368d_4.9.2-r193)) #1 SMP Sun Sep 9 02:18:33 PDT 2018


In [0]:
# Import

from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import time
import datetime as dt
import os.path
import io

from selenium import webdriver

import os
os.environ["PATH"] += os.pathsep + "."

%matplotlib inline

In [4]:
# Mount Google Drive locally

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Consolidated Functions (again by https://github.com/vincentropy/python_cartography_tutorial)

from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters

London = [51.506949, -0.122876]

def get_kernel(kernel_size, blur=1/20, halo=.001):
    """
    Create an (n*2+1)x(n*2+1) numpy array.
    Output can be used as the kernel for convolution.
    """

    # generate x and y grids
    x, y = np.mgrid[0:kernel_size*2+1, 0:kernel_size*2+1]

    center = kernel_size + 1  # center pixel
    r = np.sqrt((x - center)**2 + (y - center)**2)  # distance from center

    # now compute the kernel. This function is a bit arbitrary.
    # adjust this to get the effect you want.
    kernel = np.exp(-r/kernel_size/blur) + (1 - r/r[center,0]).clip(0)*halo
    return kernel

def add_lines(image_array, xys, width=1, weights=None):
    """
    Add a set of lines (xys) to an existing image_array
    width: width of lines
    weights: [], optional list of multipliers for lines.
    """

    for i, xy in enumerate(xys):  # loop over lines
        # create a new gray scale image
        image = Image.new("L",(image_array.shape[1], image_array.shape[0]))

        # draw the line
        ImageDraw.Draw(image).line(xy, 200, width=width)

        #convert to array
        new_image_array = np.asarray(image, dtype=np.uint8).astype(float)

        # apply weights if provided
        if weights is not None:
            new_image_array *= weights[i]

        # add to existing array
        image_array += new_image_array

    # convolve image
    new_image_array = scipy.ndimage.filters.convolve(image_array, get_kernel(width*4))
    return new_image_array

def to_image(array, hue=.62):
    """converts an array of floats to an array of RGB values using a colormap"""

    # apply saturation function
    image_data = np.log(array + 1)

    # create colormap, change these values to adjust to look of your plot
    saturation_values = [[0, 0], [.75, .68], [.78, .87], [0, 1]]
    colors = [hsv_to_rgb([hue, x, y]) for x, y in saturation_values]
    cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

    # apply colormap
    out = cmap(image_data/image_data.max())

    # convert to 8-bit unsigned integer
    out = (out*255).astype(np.uint8)
    return out


def get_min_max(bike_data):
    min_lat = bike_data["Start Station Latitude"].min()
    max_lat = bike_data["Start Station Latitude"].max()
    max_lon = bike_data["Start Station Longitude"].max()
    min_lon = bike_data["Start Station Longitude"].min()
    return min_lat, max_lat, min_lon, max_lon


def latlon_to_pixel(lat, lon, image_shape, bounds):
    min_lat, max_lat, min_lon, max_lon = bounds

    # longitude to pixel conversion (fit data to image)
    delta_x = image_shape[1]/(max_lon-min_lon)

    # latitude to pixel conversion (maintain aspect ratio)
    delta_y = delta_x/np.cos(lat/360*np.pi*2)
    pixel_y = (max_lat-lat)*delta_y
    pixel_x = (lon-min_lon)*delta_x
    return (pixel_y,pixel_x)


def row_to_pixel(row,image_shape, columns = None):
    """
    convert a row (1 trip) to pixel coordinates
    of start and end point
    """
    start_y, start_x = latlon_to_pixel(row["Start Station Latitude"],
                                       row["Start Station Longitude"], image_shape)
    end_y, end_x = latlon_to_pixel(row["End Station Latitude"],
                                   row["End Station Longitude"], image_shape)
    xy = (start_x, start_y, end_x, end_y)
    return xy


def plot_station_counts(trip_counts, zoom_start=13):
    # generate a new map
    folium_map = folium.Map(location=London,
                            zoom_start=zoom_start,
                            tiles="CartoDB dark_matter",
                            width="100%")

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = (row["Departure Count"]-row["Arrival Count"])

        # generate the popup message that is shown on click.
        popup_text = "{}<br> total departures: {}<br> total arrivals: {}<br> net departures: {}"
        popup_text = popup_text.format(row["Start Station Name"],
                          row["Arrival Count"],
                          row["Departure Count"],
                          net_departures)

        # radius of circles
        radius = np.abs(net_departures) / 9
        if np.isnan(radius):
          radius = 0

        # choose the color of the marker
        if net_departures>0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            #color="#E37222" # tangerine
            color='#E80018' # target red
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow
            #color="#0A8A9F" # teal
            color='#81D8D0' # tiffany blue

        # add marker to the map
        folium.CircleMarker(location=(row["Start Station Latitude"],
                                      row["Start Station Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map


def get_locations(bike_data):
    locations = bike_data.groupby("id_start").first()
    locations = locations.loc[:, ["Start Station Latitude",
                                 "Start Station Longitude",
                                 "Start Station Name"]]
    return locations


def get_trip_counts_by_TS(selected_hour, bike_data):
    # make a DataFrame with locations for each bike station

    locations = get_locations(bike_data)
    #select one time of day
    subset = bike_data[bike_data["TimeSlice"]==selected_hour]

    # count trips for each destination
    departure_counts =  subset.groupby("id_start").count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["Departure Count"]

    # count trips for each origin
    arrival_counts =  subset.groupby("id_end").count().iloc[:,[0]]
    arrival_counts.columns= ["Arrival Count"]

    #join departure counts, arrival counts, and locations
    trip_counts = departure_counts.join(locations).join(arrival_counts)
    return trip_counts

def add_alpha(image_data):
    """
    Uses the Value in HSV as an alpha channel.
    This creates an image that blends nicely with a black background.
    """

    # get hsv image
    hsv = rgb_to_hsv(image_data[:,:,:3].astype(float)/255)

    # create new image and set alpha channel
    new_image_data = np.zeros(image_data.shape)
    new_image_data[:,:,3] = hsv[:,:,2]

    # set value of hsv image to either 0 or 1.
    hsv[:,:,2] = np.where(hsv[:,:,2]>0, 1, 0)

    # combine alpha and new rgb
    new_image_data[:,:,:3] = hsv_to_rgb(hsv)
    return new_image_data


def create_image_map(image_data, bounds):
    min_lat, max_lat, min_lon, max_lon = bounds
    folium_map = folium.Map(location=London,
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='100%')

    # create the overlay
    map_overlay = add_alpha(to_image(image_data))

    # compute extent of image in lat/lon
    aspect_ratio = map_overlay.shape[1]/map_overlay.shape[0]
    delta_lat = (max_lon-min_lon)/aspect_ratio*np.cos(min_lat/360*2*np.pi)

    # add the image to the map
    img = folium.raster_layers.ImageOverlay(map_overlay,
                               bounds = [(max_lat-delta_lat,min_lon),(max_lat,max_lon)],
                               opacity = 1,
                               name = "Paths")

    img.add_to(folium_map)
    folium.LayerControl().add_to(folium_map)

    # return the map
    return folium_map


In [0]:
# Data manipulation (from map-density.ipynb)

# Load journey data

f = "/content/gdrive/My Drive/Colab Notebooks/folium/journeys.csv"
j = pd.read_csv(f)

date = j['day'].values
month = j['month'].values
year = j['year'].values
hour = j['hour'].values
minute = j['min'].values
station_start = j['id_start'].values
station_end = j['id_end'].values

# Extract weekday rows

weekday = np.zeros(len(date))
weekday[:] = np.nan
cnt = 0

for _year, _month, _date, _hour, _minute in zip(year, month, date, hour, minute):
  _dt = dt.datetime(_year, _month, _date, _hour, _minute)
  _weekday = _dt.weekday()
  weekday[cnt] = _weekday
  cnt += 1

IsWeekday = weekday < 5
j['IsWeekday'] = IsWeekday

#j['TimeSlice'] = (hour*3 + np.floor(minute/20)).astype(int)
j['TimeSlice'] = (hour).astype(int)
# Go for 24 TimeSlices instead for smoother interpolation

# Load Station Data

f = "/content/gdrive/My Drive/Colab Notebooks/folium/stations.csv"
stations = pd.read_csv(f)
station_id = stations['station_id'].values
stations = stations.set_index('station_id')

# Extract valid rows

valid = np.zeros(len(date))
valid[:] = False
cnt = 0

for _start, _end in zip(station_start, station_end):
  if np.logical_and((_start in station_id), (_end in station_id)):
    valid[cnt] = True
  cnt += 1

j['Valid'] = valid

# See how many journeys are valid and on weekdays

df = j[j["IsWeekday"] == True].drop(columns="IsWeekday")
df = df[df["Valid"] == True].drop(columns="Valid")

# Clean data

df = df.loc[:, ['TimeSlice', 'id_start', 'id_end', 'duration']]

In [7]:
# Add additional columns to match with the requirements of pre-defined functions

df["Start Station Latitude"] = stations.loc[df["id_start"], 'lat'].values
df["Start Station Longitude"] = stations.loc[df["id_start"], 'lon'].values
df["End Station Latitude"] = stations.loc[df["id_end"], 'lat'].values
df["End Station Longitude"] = stations.loc[df["id_end"], 'lon'].values
df["Start Station Name"] = stations.loc[df["id_start"], 'station_name'].values

df

,TimeSlice,id_start,id_end,duration,Start Station Latitude,Start Station Longitude,End Station Latitude,End Station Longitude,Start Station Name
0,17,251,478,2040.0,51.518908,-0.079249,51.521889,-0.046670,"Brushfield Street, Liverpool Street"
1,14,550,122,1800.0,51.521564,-0.039264,51.521113,-0.078869,"Harford Street, Mile End"
2,16,212,639,1140.0,51.506584,-0.199004,51.483570,-0.202038,"Campden Hill Road, Notting Hill"
3,12,163,755,420.0,51.493184,-0.167894,51.485121,-0.174971,"Sloane Avenue, Knightsbridge"
4,19,36,605,1200.0,51.501737,-0.184980,51.520331,-0.163667,"De Vere Gardens, Kensington"
5,14,589,514,1320.0,51.487196,-0.179369,51.516204,-0.155525,"Drayton Gardens, West Chelsea"
8,13,153,367,540.0,51.511933,-0.174411,51.517372,-0.164207,"Bayswater Road, Hyde Park"
9,15,396,350,960.0,51.518090,-0.163609,51.501715,-0.179854,"Shouldham Street, Marylebone"
10,8,732,298,420.0,51.506304,-0.087262,51.502279,-0.074189,"Duke Street Hill, London Bridge"
11,17,818,796,1620.0,51.503127,-0.078655,51.524000,-0.126409,"One Tower Bridge, Bermondsey"


In [0]:
# Graph interpolation (again by https://github.com/vincentropy/python_cartography_tutorial/blob/master/part2_custom_layers.ipynb)

def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)


def get_trip_counts_by_minute(float_hour, data):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    columns = ["Start Station Latitude",
               "Start Station Longitude",
               "Departure Count",
               "Arrival Count"]
    df1 = get_trip_counts_by_TS(int(float_hour), data)
    df2 = get_trip_counts_by_TS(int(float_hour) + 1, data)
    
    df = interpolate(df1.loc[:,columns], 
                     df2.loc[:,columns], 
                     float_hour % 1)
    
    df["Start Station Name"] = df1["Start Station Name"]
    return df

In [9]:
# Try generating a single map

plot_station_counts(get_trip_counts_by_minute(9.5, df), zoom_start=13)

In [0]:
# Using chromedriver in Colab

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/folium/chromedriver /usr/local/bin

In [11]:
!ls -l /usr/local/bin/chromedriver

-rw------- 1 root root 11916584 Jan  6 13:26 /usr/local/bin/chromedriver


In [0]:
# Give permission

!chmod 755 /usr/local/bin/chromedriver

In [13]:
!ls -l /usr/local/bin/chromedriver

-rwxr-xr-x 1 root root 11916584 Jan  6 13:26 /usr/local/bin/chromedriver


In [0]:
# Function for generating a frame

def go_arrivals_frame(i, hour_of_day):
    
    # create the map object
    data = get_trip_counts_by_minute(hour_of_day, df)
    my_frame = plot_station_counts(data, zoom_start = 13)
    
    # generate the png file as a byte array (NEW!!!)
    
    delay=5
    fn="frame_{:0>5}".format(i)
    DIR = '/usr/local/bin/frames'
    f = DIR + fn + '.html'
    tmpurl='file://'+DIR+'/{mapfile}'.format(path=,mapfile=fn)
    my_frame.save(f)

    browser = webdriver.Chrome()
    browser.get('file://'+f)
    #Give the map tiles some time to load
    time.sleep(delay)
    browser.save_screenshot(f+'.png')
    browser.quit()
    
    #  now add a caption to the image to indicate the time-of-day.
    hour = int(hour_of_day)
    minutes = int((hour_of_day % 1) * 60)
    
    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    
    # load a font
    font = ImageFont.truetype("Roboto-Light.ttf", 30)
    
    # draw time of day text
    draw.text((20,image.height - 50), 
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(255, 255, 255), 
              font=font)
    
    # draw title
    draw.text((image.width - 400,20), 
              "Net Arrivals vs Time of Day",
              fill=(255, 255, 255), 
              font=font)
    
    # write to a png file
    filename = f+'.png'
    image.save(filename, "PNG")
    return image

In [0]:
# Generate frames

for i, hour in enumerate(np.arange(0, 23, .1)):
    go_arrivals_frame(i, hour)